### PLAN
1. Load up to Jun 2022 into BigQuery
2. Create github action to run BQ insert script nightly
3. Set grouped database table (history_df) in BigQuery to refresh nightly after insert
4. Make script to pull data history_df from BigQuery data

8. For streamlit app append [history_df, current_df, future_df] to make the combo_df
    - put history_df into cache using st.experimental_memo so its only called once each time app is opened

In [130]:
# Initialize python libs & SQL creads
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import pandas as pd
from pandasql import sqldf
import streamlit as st
import pull_nrg_data
import json
import http.client
import certifi
import ssl
import os
import timeit
import pickle
import altair as alt

In [ ]:
pull_nrg_data.release_token(accessToken)

In [159]:
# Release token
import certifi
import ssl
import http.client
server = 'api.nrgstream.com'

def release_token(accessToken):
    path = '/api/ReleaseToken'
    headers = {'Authorization': f'Bearer {accessToken}'}
    context = ssl.create_default_context(cafile=certifi.where())
    conn = http.client.HTTPSConnection(server,context=context)
    conn.request('DELETE', path, None, headers)
    res = conn.getresponse()
    print('token released')

with open('./access_token.pickle', 'rb') as handle:
        accessToken = pickle.load(handle)
release_token(accessToken)

token released


In [124]:

with open('./access_token.pickle', 'rb') as handle:
        accessToken = pickle.load(handle)
accessToken

'7XUv4MgIEjcjmougD3vJWlIRYg3W6AM0pSj-C5K6344AqYoGS5qRnO0P9lr1pwvGf09U-QAj03xGm6FmwlkVN36AKSCXLqUv9vzBMjgQvaW5jXdE1ECCISYg3y9JW8stb6y0wj7RLPxdhqM9Mi8wIt-0U0wyLOKUPAd5KwaZusJJ55LCeTKpVDDst-YykrjA0epepThvEOU3lV3tdkZN1A61koUgie75mVgj-m9DgNLn9SAZzd-uHJOYWJNmGMDxvq2o-d1qB5OvShm8PtSry3_CPuNnLkxjSesw-pTJHxjWt5OhQyrs0ftA5JpQEKPMpPGpBLEqx79CVJByAzrrB7AhGws'

In [ ]:
# Joining daily to hourly data
import pandas as pd

intraday = pd.DataFrame(index=pd.date_range('2016-01-01', '2016-01-07', freq='H'),data=[i for i in range(145)], columns=['hourly'])
daily = pd.DataFrame(index=pd.date_range('2016-01-01', '2016-01-07', freq='D'), data=[i for i in range(7)], columns=['daily'])
df = intraday.join(daily).fillna(method='ffill')
intraday, daily, df

In [97]:
with open('./intertie.pickle', 'rb') as handle:
        intertie = pickle.load(handle)
with open('./current_outage_df.pickle', 'rb') as handle:
        outages = pickle.load(handle)

In [38]:
with open('./alerts.pickle', 'wb') as handle:
    pickle.dump(alerts, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [155]:
a = outages.set_index('timeStamp').groupby('fuelType').resample('H')['value'].ffill().reset_index()
b = pd.concat([a,intertie], axis=0)

a

,fuelType,timeStamp,value
0,Biomass & Other,2022-01-01 00:00:00,50
1,Biomass & Other,2022-01-01 01:00:00,50
2,Biomass & Other,2022-01-01 02:00:00,50
3,Biomass & Other,2022-01-01 03:00:00,50
4,Biomass & Other,2022-01-01 04:00:00,50
...,...,...,...
181059,Wind,2024-07-31 20:00:00,10
181060,Wind,2024-07-31 21:00:00,10
181061,Wind,2024-07-31 22:00:00,10
181062,Wind,2024-07-31 23:00:00,10


In [126]:
a = intertie.groupby(pd.Grouper(key='timeStamp',axis=0,freq='D')).min()
a

,value,fuelType
timeStamp,,
2022-01-01,750,Intertie
2022-01-02,750,Intertie
2022-01-03,416,Intertie
2022-01-04,485,Intertie
2022-01-05,347,Intertie
...,...,...
2023-08-17,750,Intertie
2023-08-18,750,Intertie
2023-08-19,750,Intertie


datetime.datetime(2022, 12, 31, 13, 42, 7, 702777)